# Analyzing the data

In [75]:
import pandas as pd
import numpy as np

# data 5 is bad and 9 is largest

df = pd.read_csv("odds_datas/test_5_odds_data.csv") # odds_data
df["T_goals"] = df["H_score"] + df["A_score"]
df

,week,HT,HT.1,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score,T_goals
0,17,LIV,CRY,1.77,3.33,5.32,1.22,4.33,1.74,2.09,2.87,1.42,6.36,1.12,0,0,0
1,17,MNC,FOR,1.29,5.52,10.10,1.15,5.34,1.53,2.50,2.45,1.55,4.79,1.18,1,0,1
2,17,CHE,TOT,2.24,3.07,3.57,1.24,4.08,1.82,1.98,3.08,1.37,7.15,1.09,3,0,3
3,17,ASV,WOL,1.86,3.42,4.51,1.25,3.94,1.83,1.97,3.17,1.36,7.11,1.09,1,1,2
4,17,BUR,BOU,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,34,CHE,WOL,1.86,3.41,4.51,1.23,4.14,1.79,2.03,3.03,1.38,6.76,1.10,2,1,3
1316,34,BRN,TOT,2.43,3.23,3.02,1.28,3.67,1.93,1.87,3.43,1.31,7.95,1.08,0,1,1
1317,34,LIV,NWC,2.25,3.04,3.59,1.23,4.21,1.78,2.03,2.96,1.40,6.79,1.10,1,1,2
1318,34,EVE,FUL,2.29,3.02,3.53,1.33,3.35,2.07,1.75,3.76,1.27,9.27,1.06,2,2,4


In [77]:
# count = 1
# martingale_list = []
# for x in odd_list:
#     if x[1] > 3:
#         # print(count)
#         martingale_list.append(count)
#         count = 1             
#     else:
#         count += 1

# print("Total Wins:", len(martingale_list))
# print(martingale_list)
# print(sorted(martingale_list, reverse=True)[:50])
# print("Hightest Martingale Level:", max(martingale_list))

Total Wins: 39
[1, 9, 1, 6, 9, 2, 4, 1, 7, 2, 3, 1, 1, 1, 3, 8, 4, 6, 1, 2, 3, 3, 3, 6, 2, 2, 2, 2, 4, 1, 14, 4, 1, 1, 2, 1, 1, 1, 4]
[14, 9, 9, 8, 7, 6, 6, 6, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Hightest Martingale Level: 14


# STRATEGIES

In [243]:
import pandas as pd
import numpy as np

# data 5 is baddest and 9 is largest

df = pd.read_csv("odds_datas/test_16_odds_data.csv") # odds_data
df["T_goals"] = df["H_score"] + df["A_score"]

check_list = []
odd_list = []
for _, row in df.iterrows():
    check_list.append((row["week"], len(check_list)+1, row["over_3"], row["H_score"], row["A_score"]))
    if len(check_list) == 10:
        # print(check_list)
        sorted_odds = sorted(check_list, key=lambda x: x[2], reverse=True)
        all_odds = [x[2] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[2]) == 1:
                odd_list.append(odd)
                break               
        check_list = []

new_df = pd.DataFrame(odd_list, columns=["week", "odd_position_num", "odd", "H_score", "A_score"])
print("Total Stake: ", len(new_df))

print("Mean Odd:", np.mean(new_df["odd"]))
new_df.head(3)

Total Stake:  61
Mean Odd: 3.7057377049180324


,week,odd_position_num,odd,H_score,A_score
0,15,8,3.43,0,2
1,16,3,3.83,3,2
2,17,4,3.83,3,1


## Martingale Strategy

In [245]:

bankroll = 100000
loss_amount = 3700
highest_loss = -(loss_amount * 3)
print("Highest Loss :", highest_loss)

mart_single_target_prtg = 1
mart_single_target = (mart_single_target_prtg * bankroll) / 100
print("Single Target -", mart_single_target)

total_target_prtg = 3
total_target = (total_target_prtg * bankroll) / 100
print("Total Target -", total_target, "\n")


single_bankroll = 0
single_stake = 1000
single_bankroll_list = []

martingale_bankroll = 0
total_loss = 0
martingale_bankroll_list = []
mart_counter = 0
single_counter = 0

# odd_df = new_df.copy()
odd_df = pd.read_csv("auto_trade_records/rec_1.csv") 

odd_df["T_goals"] = odd_df["H_score"] + odd_df["A_score"]
for _, row in odd_df.iterrows():
    if row["T_goals"] > 3:
        if single_counter <= 5:
            single_bankroll += (row["odd"] - 1) * single_stake
        single_counter = 0
        if mart_counter <= 5:
            martingale_bankroll += mart_single_target 
        mart_counter = 0
    else:
        single_counter += 1
        if single_counter <= 5:
            single_bankroll -= single_stake
        mart_counter += 1
        if mart_counter == 5:
            martingale_bankroll -= loss_amount
    martingale_bankroll_list.append(int(martingale_bankroll))
    single_bankroll_list.append(int(single_bankroll))

odd_df["Single_Bankroll"] = single_bankroll_list
odd_df["Check_single"] = np.where(odd_df["Single_Bankroll"] >= total_target, "Achieved", "-")

odd_df["Martingale_Bankroll"] = martingale_bankroll_list    
odd_df["Check_Martingale"] = np.where(odd_df["Martingale_Bankroll"] >= total_target, "Achieved", "-")   
print("TOTAL GAMES:", len(odd_df))


mart_targets = odd_df[odd_df["Check_Martingale"] == "Achieved"]
single_targets = odd_df[odd_df["Check_single"] == "Achieved"]

mart_single_list = [mart_targets, single_targets]

for temp in range(len(mart_single_list)):
    val = "Martingale_Bankroll" if temp == 0 else "Single_Bankroll"
    if len(mart_single_list[temp]) > 0:
        print("\n--->", val.upper(), "STRATEGY <---")
        print("     Target Amount:", mart_single_list[temp][val].iloc[0])
        print("     Target Position:", (mart_single_list[temp].head(1).index[0]) + 1)

        # Highest Profit
        print("    ", "-" * 20)
        print("     Highest Profit:", mart_single_list[temp][val].max())
        print("     Highest Profit POSITION:", (odd_df[odd_df[val] == mart_single_list[temp][val].max()].index[0]) + 1, "\n")


Highest Loss : -11100
Single Target - 1000.0
Total Target - 3000.0 

TOTAL GAMES: 168

---> MARTINGALE_BANKROLL STRATEGY <---
     Target Amount: 3000
     Target Position: 5
     --------------------
     Highest Profit: 17600
     Highest Profit POSITION: 75 


---> SINGLE_BANKROLL STRATEGY <---
     Target Amount: 4610
     Target Position: 3
     --------------------
     Highest Profit: 21390
     Highest Profit POSITION: 75 



In [230]:
# For 3% Target
martingale_strategy = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
single_stake_strategy = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
better_performance = ["S", "S", "S", "M", "S", "M", "S", "S", "S", "S", "S", "S", "M", "", "S", "S"]


In [242]:
odd_df.head(50)

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
0,20,8,3.85,3,1,4,2850,-,1000,-
1,21,5,3.76,1,0,1,1850,-,1000,-
2,22,6,3.76,1,0,1,850,-,1000,-
3,23,6,3.88,4,1,5,3730,Achieved,2000,-
4,24,10,3.76,1,2,3,2730,-,2000,-
5,25,1,3.88,1,1,2,1730,-,2000,-
6,26,8,3.85,3,1,4,4580,Achieved,3000,Achieved
7,27,1,3.85,1,1,2,3580,Achieved,3000,Achieved
8,28,4,3.76,2,1,3,2580,-,3000,Achieved
9,29,6,3.85,1,0,1,1580,-,3000,Achieved


In [232]:
odd_df[50: 100]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
50,26,8,3.88,0,0,0,-6680,-,-4800,-
51,27,7,3.85,1,0,1,-6680,-,-4800,-
52,28,3,3.85,2,1,3,-6680,-,-4800,-
53,29,2,3.76,1,0,1,-6680,-,-4800,-
54,30,10,3.76,3,0,3,-6680,-,-4800,-
55,31,5,3.85,2,2,4,-6680,-,-4800,-
56,32,3,3.58,1,1,2,-7680,-,-4800,-
57,33,2,3.76,0,3,3,-8680,-,-4800,-
58,34,9,3.42,1,1,2,-9680,-,-4800,-
59,35,1,3.76,0,2,2,-10680,-,-4800,-


In [233]:
odd_df[100: 150]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
100,38,4,3.85,1,2,3,-5630,-,-1200,-
101,1,5,3.85,3,0,3,-6630,-,-1200,-
102,2,7,3.85,1,1,2,-7630,-,-1200,-
103,3,9,3.53,2,1,3,-8630,-,-1200,-
104,4,9,3.83,2,0,2,-9630,-,-4900,-
105,5,10,3.76,1,0,1,-9630,-,-4900,-
106,6,2,3.85,1,0,1,-9630,-,-4900,-
107,7,5,3.76,0,0,0,-9630,-,-4900,-
108,8,4,3.88,3,0,3,-9630,-,-4900,-
109,9,10,3.85,1,1,2,-9630,-,-4900,-


In [203]:
odd_df[150: 200]

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale


In [176]:

count = 1
martingale_list = []
for x in odd_df["T_goals"]:
    if x > 3:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print(martingale_list)
print(sorted(martingale_list, reverse=True)[:50])
print("Hightest Martingale Level:", max(martingale_list))


single_target = 1000
loss_amount = 3700

win_length = sorted(martingale_list, reverse=True)
total_diff = 0
for x in win_length:
    if x > 5:
        total_diff += 1
    else:
        break
print("\nTotal Played:", len(martingale_list))
print("Total Lost:", total_diff)
won = len(martingale_list) - total_diff
print("Total Won:", won)
print("Profit/Loss Amount:", (won * single_target) - (total_diff * loss_amount))

[1, 4, 2, 4, 10, 1, 11, 2, 7, 3, 2, 6, 8, 6, 1, 8, 1, 3, 3, 1, 2, 2, 2, 15, 1, 3, 3, 3, 6, 1, 4, 1, 1, 1, 3, 4, 3, 2, 1, 2, 3, 4, 6, 1, 2, 1, 4, 9, 2, 2, 2, 1, 1, 2, 3, 9, 31, 4, 1, 4, 3, 3, 9, 4, 14, 4, 1, 1, 1, 2, 6, 3]
[31, 15, 14, 11, 10, 9, 9, 9, 8, 8, 7, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Hightest Martingale Level: 31

Total Played: 72
Total Lost: 16
Total Won: 56
Profit/Loss Amount: -3200


## Single Stake Strategy

In [154]:
targets = odd_df[odd_df["Check_single"] == "Achieved"]
print("Total Games:", len(odd_df))
print("Target Position:", (targets.head(1).index[0]) + 1)

# Highest Profit
print("Highest Profit:", targets["Single_Bankroll"].max())
print("Highest Profit POSITION:", (odd_df[odd_df["Single_Bankroll"] == targets["Single_Bankroll"].max()].index[0]) + 1)

Total Games: 27
Target Position: 26
Highest Profit: 6820
Highest Profit POSITION: 26


In [155]:
targets

,week,odd_position_num,odd,H_score,A_score,T_goals,Single_Bankroll,Check_single,Martingale_Bankroll,Check_Martingale
25,8,8,3.85,4,2,6,6820,Achieved,-18500,-
26,9,9,3.42,0,1,1,5820,Achieved,-18500,-
